In [1]:
import main
import plot_inequality as plot_ineq
import time
import importlib
import io_manager
import numpy as np
importlib.reload(plot_ineq)
#run stuff here
start_time = time.perf_counter()
print("Running main")

of_name = None
main_path = "C:/Users/Ben/My Drive/PhD/PhD Year 3/3rd Year Paper/Model/My Code/MH_Model/my_code/model_uncert/"
input_path = main_path + "input/50p_age_moms/"

trans_path = main_path + "input/50p_age_moms/MH_trans_uncond_age.csv"
test_trans_path = main_path + "input/50p_age_moms/MH_trans_test_age.csv"
iid_trans_path = main_path + "input/50p_age_moms/MH_trans_test_iid.csv"
perm_trans_path = main_path + "input/50p_age_moms/MH_trans_test_perm.csv"

do_dpi_calib = False
output_flag = False
myPars, myShocks, sols, sims = main.main_io(main_path, out_folder_name = of_name, H_trans_uncond_path = trans_path,
                                            output_flag = output_flag, do_dpi_calib = do_dpi_calib)

Running main
*****Running main_io with default out_folder_name*****
Solver ran in 7.891445200002636 seconds
Calibrating with alpha_lab_targ = 0.33495033, w0_mean_targ = 2.1986368, w0_sd_targ = 0.29955208, 
                                        w1_targ = 0.2915730000000001, w2_targ = 0.2731836000000003, wH_targ = 0.051262233,
                                        dpi_BB_targ = 0.50886095, dpi_GG_targ = 0.34358001
***** Calibration iteration 0 *****
Calibrating w0_mu
Calibrating w0_sigma
***** Calibration iteration 1 *****
Calibrating w0_mu
Calibrating w0_sigma
***** Calibration iteration 2 *****
Calibrating w0_mu
Calibrating w0_sigma
Calibrating w1
Calibrating w2
***** Calibration iteration 3 *****
Calibrating w0_mu
Calibrating w0_sigma
Calibrating w1
Calibrating w2
***** Calibration iteration 4 *****
Calibrating w0_mu
Calibrating w0_sigma
Calibrating w1
Calibrating w2
***** Calibration iteration 5 *****
Calibrating w0_mu
Calibrating w0_sigma
Calibrating w1
Calibrating w2
***** Cali

In [2]:
from pars_shocks import Pars, Shocks
import pars_shocks

myPars_H_good: Pars = main.pars_factory(main_path, H_trans_uncond_path = trans_path)
myShocks_H_good: Shocks = Shocks(myPars_H_good)

H_hist = np.ones(myPars.state_space_shape_sims, dtype=np.int64)
myShocks_H_good.H_hist = H_hist
print("np.mean(H_hist)", np.mean(H_hist))

myPars_H_good, myShocks_H_good, sols_H_good, sims_H_good = main.main_io(main_path, out_folder_name = of_name, H_trans_uncond_path = trans_path,
                                                                    myPars = myPars_H_good, myShocks=myShocks_H_good, 
                                                                    output_flag = False, do_wH_calib = False, do_dpi_calib = False)

myPars_no_wH: Pars = main.pars_factory(main_path, H_trans_uncond_path = trans_path)
myPars_no_wH.wH_coeff = 0.0
myPars_no_wH, myShocks_no_wH, sols_no_wH, sims_no_wH = main.main_io(main_path, out_folder_name = of_name, H_trans_uncond_path = trans_path,
                                                                    myPars = myPars_no_wH, myShocks=myShocks, 
                                                                    output_flag = False, do_wH_calib = False, do_dpi_calib = False)

np.mean(H_hist) 1.0
*****Running main_io with default out_folder_name*****
Solver ran in 0.5218080000049667 seconds
Calibrating with alpha_lab_targ = 0.33495033, w0_mean_targ = 2.1986368, w0_sd_targ = 0.29955208, 
                                        w1_targ = 0.2915730000000001, w2_targ = 0.2731836000000003, wH_targ = 0.051262233,
                                        dpi_BB_targ = 0.50886095, dpi_GG_targ = 0.34358001
***** Calibration iteration 0 *****
Calibrating w0_mu
Calibrating w0_sigma
***** Calibration iteration 1 *****
Calibrating w0_mu
Calibrating w0_sigma
***** Calibration iteration 2 *****
Calibrating w0_mu
Calibrating w0_sigma
Calibrating w1
Calibrating w2
***** Calibration iteration 3 *****
Calibrating w0_mu
Calibrating w0_sigma
Calibrating w1
Calibrating w2
***** Calibration iteration 4 *****
Calibrating w0_mu
Calibrating w0_sigma
Calibrating w1
Calibrating w2
***** Calibration iteration 5 *****
Calibrating w0_mu
Calibrating w0_sigma
Calibrating w1
Calibrating w2
**

In [33]:
import model_uncert as model
from typing import Tuple
age_25_ind = np.where(myPars.age_grid == 25)[0][0] 
age_55_ind = np.where(myPars.age_grid == 55)[0][0]

lab_earn_trim: np.array = sims["lab_earnings"][:, :, :, age_25_ind:age_55_ind+1]*12
lab_earn_trim_no_wH: np.array = sims_no_wH["lab_earnings"][:, :, :, age_25_ind:age_55_ind+1]*12
lab_earn_trim_H_good: np.array = sims_H_good["lab_earnings"][:, :, :, age_25_ind:age_55_ind+1]*12

log_lab_earn = np.log(lab_earn_trim)
log_lab_earn_no_wH = np.log(lab_earn_trim_no_wH)
log_lab_earn_H_good = np.log(lab_earn_trim_H_good)



In [34]:


def calc_wstats(earnings: np.array, weights_lab_fe: np.array, weight_H_type: np.array) -> dict:

    weights = np.outer(weights_lab_fe, weight_H_type)
    sim_num = earnings.shape[2]
    age_num = earnings.shape[3]
    weights_exp = np.repeat(weights[:, :, np.newaxis], sim_num, axis=2)
    weights_exp = np.repeat(weights_exp[:, :, :, np.newaxis], age_num, axis=3)

    flat_earns = earnings.flatten()
    flat_weights = weights_exp.flatten()

    mean_earn = np.average(flat_earns, weights=flat_weights)

    #calculate the weighted median
    sorted_inds = np.argsort(flat_earns)
    sorted_earns = flat_earns[sorted_inds]
    sorted_weights = flat_weights[sorted_inds]
    cum_weights = np.cumsum(sorted_weights)
    cutoff = 0.5 * cum_weights[-1]
    median_earn = sorted_earns[np.searchsorted(cum_weights, cutoff)]

    #calc weighted variance
    var_earn = np.average((flat_earns - mean_earn)**2, weights=flat_weights)
    #calc weighted std dev
    std_earn = np.sqrt(var_earn)

    #calc weighted min and max
    min_earn = np.min(flat_earns)
    max_earn = np.max(flat_earns)

    # Calculate weighted percentiles
    pct_10 = np.percentile(sorted_earns, 10)
    pct_25 = np.percentile(sorted_earns, 25)
    pct_75 = np.percentile(sorted_earns, 75)
    pct_90 = np.percentile(sorted_earns, 90)

    # Step 5: Return the results in a dictionary
    ret_dict = {
        "mean": mean_earn,
        "median": median_earn,
        "var": var_earn,
        "std_dev": std_earn,
        "min": min_earn,
        "max": max_earn,
        "P10": pct_10,
        "P25": pct_25,
        "P75": pct_75,
        "P90": pct_90
    }

    return ret_dict


In [35]:
earnings = lab_earn_trim 
weights_lab_fe = myPars.lab_fe_weights
weight_H_type = myPars.H_type_perm_weights
wstats = calc_wstats(earnings, weights_lab_fe, weight_H_type)
for key in wstats:
    print(f"{key}: {wstats[key]}")
print("\n")

earnings_no_wH = lab_earn_trim_no_wH 
weights_lab_fe = myPars_no_wH.lab_fe_weights
weight_H_type = myPars_no_wH.H_type_perm_weights
wstats_no_wH = calc_wstats(earnings_no_wH, weights_lab_fe, weight_H_type)
for key in wstats_no_wH:
    print(f"{key}: {wstats_no_wH[key]}")

mean: 19351.178087521315
median: 18101.979653097045
var: 57278208.346111186
std_dev: 7568.236805631229
min: 4815.214547841646
max: 55526.37149400996
P10: 11694.946374019819
P25: 16322.444429427633
P75: 35505.88089104486
P90: 44131.87744888323


mean: 19367.124475731733
median: 18544.8651585013
var: 55527058.17159748
std_dev: 7451.648017156841
min: 5265.133630251448
max: 49951.18252640851
P10: 11584.01197346346
P25: 16290.801251072602
P75: 34114.34158018045
P90: 42310.11399042532


In [36]:

import numpy as np

def calculate_weighted_lt_stats(earnings: np.array, weights_lab_fe: np.array, weights_H_type: np.array) -> dict:
    # Step 1: Calculate lifetime earnings (annualized mean earnings from ages 25-55)
    # Average across the last axis (age), assuming the earnings matrix is already restricted to ages 25-55.
    lifetime_earnings = earnings.mean(axis=-1)  # Shape: (lab_fe, H_type, simulation_number)
    
    # Step 2: Expand the weights to match the dimensions of lifetime_earnings
    # Create a grid of weights by multiplying the lab_fe and H_type weights
    weights = np.outer(weights_lab_fe, weights_H_type)  # Shape: (lab_fe, H_type)
    
    # Reshape the weights to broadcast over the (simulation_number) dimension
    sim_num = lifetime_earnings.shape[2]
    weights_expanded = np.repeat(weights[:, :, np.newaxis], sim_num, axis=2)  # Shape: (lab_fe, H_type, simulation_number)

    # Step 3: Flatten the earnings and weights for weighted calculation
    flattened_earnings = lifetime_earnings.flatten()  # Shape: (lab_fe * H_type * simulation_number,)
    flattened_weights = weights_expanded.flatten()    # Shape: (lab_fe * H_type * simulation_number,)

    # Step 4: Compute weighted statistics

    # Calculate weighted mean
    mean_earn = np.average(flattened_earnings, weights=flattened_weights)

    # Calculate weighted median
    sorted_indices = np.argsort(flattened_earnings)
    sorted_earnings = flattened_earnings[sorted_indices]
    sorted_weights = flattened_weights[sorted_indices]
    cumsum_weights = np.cumsum(sorted_weights)
    cutoff = 0.5 * cumsum_weights[-1]
    median_earn = sorted_earnings[np.searchsorted(cumsum_weights, cutoff)]

    # Calculate weighted variance
    var_earn = np.average((flattened_earnings - mean_earn) ** 2, weights=flattened_weights)

    # Calculate weighted standard deviation
    std_earn = np.sqrt(var_earn)

    # Calculate weighted min and max
    min_earn = np.min(flattened_earnings)
    max_earn = np.max(flattened_earnings)

    # Calculate weighted percentiles
    pct_10 = np.percentile(sorted_earnings, 10)
    pct_25 = np.percentile(sorted_earnings, 25)
    pct_75 = np.percentile(sorted_earnings, 75)
    pct_90 = np.percentile(sorted_earnings, 90)

    # Step 5: Return the results in a dictionary
    ret_dict = {
        "mean": mean_earn,
        "median": median_earn,
        "var": var_earn,
        "std_dev": std_earn,
        "min": min_earn,
        "max": max_earn,
        "P10": pct_10,
        "P25": pct_25,
        "P75": pct_75,
        "P90": pct_90
    }
    
    return ret_dict





In [37]:
earnings = lab_earn_trim 
weights_lab_fe = myPars.lab_fe_weights
weights_H_type = myPars.H_type_perm_weights
lt_stats = calculate_weighted_lt_stats(earnings, weights_lab_fe, weights_H_type)
for key in lt_stats:
    print(key, lt_stats[key])

print()

earnings_no_wH = lab_earn_trim_no_wH 
weights_lab_fe = myPars_no_wH.lab_fe_weights
weights_H_type = myPars_no_wH.H_type_perm_weights
lt_stats_no_wH = calculate_weighted_lt_stats(earnings_no_wH, weights_lab_fe, weights_H_type)
for key in lt_stats_no_wH:
    print(key, lt_stats_no_wH[key])

print()

earnings_H_good = lab_earn_trim_H_good
weights_lab_fe = myPars_H_good.lab_fe_weights
weights_H_type = myPars_H_good.H_type_perm_weights
lt_stats_H_good = calculate_weighted_lt_stats(earnings_H_good, weights_lab_fe, weights_H_type)
for key in lt_stats_H_good:
    print(key, lt_stats_H_good[key])



mean 19351.178087521308
median 17773.922744430434
var 35272063.55324185
std_dev 5939.0288392330485
min 10112.416323497946
max 43912.42010557377
P10 13246.942277782873
P25 17612.85297084476
P75 35223.75900471892
P90 40239.63113491836

mean 19367.124475731733
median 17108.073594104753
var 35280002.21683122
std_dev 5939.697148578472
min 10264.844156462843
max 41059.376625851415
P10 13344.297403402015
P25 17108.073594104753
P75 34216.14718820952
P90 37979.92337892002

mean 19708.041458216943
median 19176.68902241522
var 32395677.66720341
std_dev 5691.72009740495
min 9786.64656019377
max 56104.44009681027
P10 13646.86164627688
P25 20011.970851538634
P75 39804.029679350264
P90 45909.15767750197


In [38]:
var_earn = lt_stats["var"]
var_earn_H = lt_stats_H_good["var"]
print(f"var_earn: {round(var_earn/1000, 2)}")
print(f"var_earn_H: {round(var_earn_H/1000, 2)}")
print(f"liftetime earn var %: {(var_earn - var_earn_H)/var_earn*100}")
print(f"log lifetime earn var %: {(np.log(var_earn) - np.log(var_earn_H))*100}")

var_earn: 35272.06
var_earn_H: 32395.68
liftetime earn var %: 8.154855702436127
log lifetime earn var %: 8.506624129255869


In [39]:
import pandas as pd
import my_toolbox as tb
outpath = myPars.path + "output/"

lt_stats_data_path = input_path + "lt_earn_stats.csv" 
lt_stats_df =  pd.read_csv(lt_stats_data_path)
# change the column names
lt_stats_df.columns = ["mean", "std_dev",  "P10", "P25", "P50", "P75", "P90"]
# convert to dictionary
lt_stats_data = lt_stats_df.to_dict(orient = "records")[0]
for key in lt_stats_df:
    print(key, lt_stats_df[key])

r2f =lambda x : round(x/1000, 2) 
r2 = lambda x : round(x, 2)
rf = lambda x : round(x/1000)

tab = [
    "\\documentclass[border=3mm,preview]{standalone}",
    "\\usepackage{booktabs} \n",
    "\\usepackage{caption} \n",
    "\\usepackage{pdflscape} \n",
    "\\begin{document}\n",
    "\\begin{landscape}\n",
    # "\\textit{Liftime Earnings Statistics} \\\\ \n",
    "\\small\n",
    "\\begin{table} \n",
    "\\center\\caption*{Annualized Lifetime Earnings Statistics (1000s GBP)} \n",
    "\\begin{tabular}{l | l l l | l l l l l | l l l} \n",
    "\\toprule \n",
    " & \\multicolumn{3}{c}{Summary} & \\multicolumn{5}{c}{Percentiles} & \\multicolumn{3}{c}{Ratios} \\\\ \n",
    # "\\hline \n",
    "Source & Mean & Var. & SD & 10th & 25th & 50th & 75th & 90th & 90/10 & 90/50 & 50/10 \\\\ \n", 
    "\\midrule \n",
    f"""Baseline Model & {r2f(lt_stats['mean'])} & {rf(lt_stats['var'])} & {r2f(lt_stats['std_dev'])} & 
        {r2f(lt_stats['P10'])} & {r2f(lt_stats['P25'])} & {r2f(lt_stats['median'])} & {r2f(lt_stats['P75'])} & {r2f(lt_stats['P90'])} &
        {r2(lt_stats['P90']/lt_stats['P10'])} & {r2(lt_stats['P90']/lt_stats['median'])} & {r2(lt_stats['median']/lt_stats['P10'])} \\\\ \n""",
    f"""$H = Good$ & {r2f(lt_stats_H_good['mean'])} & {rf(lt_stats_H_good['var'])} & {r2f(lt_stats_H_good['std_dev'])} & 
        {r2f(lt_stats_H_good['P10'])} & {r2f(lt_stats_H_good['P25'])} & {r2f(lt_stats_H_good['median'])} & {r2f(lt_stats_H_good['P75'])} & {r2f(lt_stats_H_good['P90'])} &
        {r2(lt_stats_H_good['P90']/lt_stats_H_good['P10'])} & {r2(lt_stats_H_good['P90']/lt_stats_H_good['median'])} 
        & {r2(lt_stats_H_good['median']/lt_stats_H_good['P10'])} \\\\ \n""",
    # f"""Data & {r2f(lt_stats_data['mean'])} & - & {r2f(lt_stats_data['std_dev'])} & 
    #     {r2f(lt_stats_data['P10'])} & {r2f(lt_stats_data['P25'])} & {r2f(lt_stats_data['P50'])} & {r2f(lt_stats_data['P75'])} & {r2f(lt_stats_data['P90'])} &
    #     {r2(lt_stats_data['P90']/lt_stats_data['P10'])} & {r2(lt_stats_data['P90']/lt_stats_data['P50'])} & {r2(lt_stats_data['P50']/lt_stats_data['P10'])} \\\\ \n""",
    "\\bottomrule \n",   
    "\\end{tabular}\n",
    "\\end{table}\n",
    "\\end{landscape}\n",
    "\\end{document}\n"
]

file_name = "lt_earnings_stats.tex"
tb.list_to_tex(outpath, file_name, tab)
tb.tex_to_pdf(outpath, file_name)


mean 0    20269.164
Name: mean, dtype: float64
std_dev 0    10871.284
Name: std_dev, dtype: float64
P10 0    8693.0332
Name: P10, dtype: float64
P25 0    12838.812
Name: P25, dtype: float64
P50 0    18598.17
Name: P50, dtype: float64
P75 0    26077.945
Name: P75, dtype: float64
P90 0    34573.301
Name: P90, dtype: float64
PDF successfully created at C:/Users/Ben/My Drive/PhD/PhD Year 3/3rd Year Paper/Model/My Code/MH_Model/my_code/model_uncert/output/lt_earnings_stats.pdf


In [40]:
earnings = log_lab_earn 
weights_lab_fe = myPars.lab_fe_weights
weights_H_type = myPars.H_type_perm_weights
lt_stats = calculate_weighted_lt_stats(earnings, weights_lab_fe, weights_H_type)
for key in lt_stats:
    print(key, lt_stats[key])

print()

earnings_no_wH = log_lab_earn_no_wH
weights_lab_fe = myPars_no_wH.lab_fe_weights
weights_H_type = myPars_no_wH.H_type_perm_weights
lt_stats_no_wH = calculate_weighted_lt_stats(earnings_no_wH, weights_lab_fe, weights_H_type)
for key in lt_stats_no_wH:
    print(key, lt_stats_no_wH[key])

print()

earnings_H_good = log_lab_earn_H_good
weights_lab_fe = myPars_H_good.lab_fe_weights
weights_H_type = myPars_H_good.H_type_perm_weights
lt_stats_H_good = calculate_weighted_lt_stats(earnings_H_good, weights_lab_fe, weights_H_type)
for key in lt_stats_H_good:
    print(key, lt_stats_H_good[key])

mean 9.79376949348782
median 9.753854626527081
var 0.08889470507151259
std_dev 0.29815215087520763
min 9.186682346794033
max 10.664110907659271
P10 9.454985695737275
P25 9.74421200627656
P75 10.43719690798779
P90 10.573338244239535

mean 9.79694636628758
median 9.717813909020784
var 0.08937560932188034
std_dev 0.29895753765690597
min 9.20698828525479
max 10.593282646374684
P10 9.465902150461421
P25 9.717813909020784
P75 10.410961089580727
P90 10.514972407084205

mean 9.771477104086953
median 9.777009926947926
var 0.07881909683607369
std_dev 0.280747389722636
min 9.114894419728023
max 10.912579643989819
P10 9.452869991518591
P25 9.823394375395939
P75 10.51353929686326
P90 10.65640195475447


In [41]:
var_earn = lt_stats["var"]
var_earn_H = lt_stats_H_good["var"]
print(f"var_earn: {var_earn}")
print(f"var_earn_H: {var_earn_H}")
print(f"liftetime earn var %: {((var_earn - var_earn_H)/var_earn_H)*100}")
print(f"log lifetime earn var %: {(np.log(var_earn) - np.log(var_earn_H))*100}")
print(f'raw diff: {var_earn - var_earn_H}')

var_earn: 0.08889470507151259
var_earn_H: 0.07881909683607369
liftetime earn var %: 12.78320691290582
log lifetime earn var %: 12.029726710731437
raw diff: 0.010075608235438904
